## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.data import Data, get_buy_and_hold_df, INDICATORS

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
    plot_avg_portfolio_allocation, 
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

In [3]:
import numpy as np 
import pandas as pd

from pickle import load, dump

### Data

#### Settings

In [74]:
# assets in the portfolio
ASSETS = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]
n_assets = 5
assets = np.random.choice(ASSETS, size=n_assets, replace=False).tolist()

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# technical indicators
momentum = [
    "MOM",
    "MACD",
    "MFI",
    "RSI"
]

volatility = [
    "ATR",
    "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"      
]

volume = [
    "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

# number of principal components (None means no PCA)
pca_ncp = 5

# boolean indicating whether to apply discrete wavelet transformation
dwt = True

#### Instantiate `Data` object

In [75]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp, dwt)
data

[*********************100%***********************]  5 of 5 completed


Data(start_date='2009-12-31', end_date='2017-12-29', window_size=100, assets=['NKE', 'XOM', 'MCD', 'AMAT', 'NOV'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'HTDCP', 'HTS', 'CO', 'OBV'], pca_ncp=5, discrete_wavelet=True)

### `RRL` - Training and validation

In [76]:
initial_invest = 100
n_epochs = 100

#### No transaction fees 

$\delta=0$

In [77]:
n_features = data.pca_ncp
# n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_no_fees_pca_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=5, n_features=5, delta=0, rho=0.1, l2=0.01)
Training window: 2010-04-01/2010-05-24


Training in progress...:  29%|██▉       | 29/100 [00:00<00:00, 74.82it/s, sharpe_ratio=-.171]


Validation window: 2010-08-24/2010-10-14
Sharpe ratio on validation set: 7.610180898799901
Cumulative profits: 116.16041131838844
Training window: 2010-08-24/2010-10-14


Training in progress...:  22%|██▏       | 22/100 [00:00<00:01, 75.56it/s, sharpe_ratio=0.477]


Validation window: 2011-01-14/2011-03-09
Sharpe ratio on validation set: 3.6102609969945187
Cumulative profits: 124.18005753604898
Training window: 2011-01-14/2011-03-09


...

Validation window: 2017-05-25/2017-07-18
Sharpe ratio on validation set: 1.5513113336942694
Cumulative profits: 138.02849137676228
Training window: 2017-05-25/2017-07-18


Training in progress...:  17%|█▋        | 17/100 [00:00<00:01, 79.98it/s, sharpe_ratio=0.0697]


Validation window: 2017-10-17/2017-12-07
Sharpe ratio on validation set: -0.5169797080080291
Cumulative profits: 136.49386307529164
Training window: 2017-10-17/2017-12-07


Training in progress...:  11%|█         | 11/100 [00:00<00:01, 64.00it/s, sharpe_ratio=-.0174]


Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [62]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_10bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_10bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...: 100%|██████████| 50/50 [00:02<00:00, 18.02it/s, sharpe_ratio=-.00879]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 3.7981108023897776
Cumulative profits: 118.90747473860743
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 50/50 [00:02<00:00, 19.71it/s, sharpe_ratio=0.221]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 2.4147199532246075
Cumulative profits: 129.68842585153604
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.1130977105185305
Cumulative profits: 164.3718972972654
Training window: 2017-03-31/2017-07-18


Training in progress...: 100%|██████████| 50/50 [00:02<00:00, 20.24it/s, sharpe_ratio=-.0135]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 4.779033895265222
Cumulative profits: 189.9167967630297
Training window: 2017-08-23/2017-12-07


Training in progress...:  46%|████▌     | 23/50 [00:01<00:01, 20.27it/s, sharpe_ratio=0.257]


Less than 30 observations in the validation set after preprocessing.


In [63]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_30bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  14%|█▍        | 7/50 [00:00<00:02, 17.44it/s, sharpe_ratio=-.0409]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.3593927917378092
Cumulative profits: 110.52273980236926
Training window: 2010-06-30/2010-10-14


Training in progress...:   2%|▏         | 1/50 [00:00<00:03, 16.04it/s, sharpe_ratio=0.149]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.005539435596743
Cumulative profits: 114.48226271025298
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.6460023869723759
Cumulative profits: 81.71681216360064
Training window: 2017-03-31/2017-07-18


Training in progress...:  10%|█         | 5/50 [00:00<00:02, 19.22it/s, sharpe_ratio=-.104]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 3.084852721257747
Cumulative profits: 89.62195362672915
Training window: 2017-08-23/2017-12-07


Training in progress...:  30%|███       | 15/50 [00:00<00:01, 20.48it/s, sharpe_ratio=0.19]


Less than 30 observations in the validation set after preprocessing.


In [64]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_50bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  74%|███████▍  | 37/50 [00:01<00:00, 21.11it/s, sharpe_ratio=-.117]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.3223130873905604
Cumulative profits: 105.6819940486498
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 50/50 [00:02<00:00, 22.28it/s, sharpe_ratio=0.0935]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: -0.05922949453324463
Cumulative profits: 105.22335285879339
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -2.6894397521279854
Cumulative profits: 40.625131298608224
Training window: 2017-03-31/2017-07-18


Training in progress...:  34%|███▍      | 17/50 [00:00<00:01, 18.40it/s, sharpe_ratio=-.171]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: -0.9949353013692226
Cumulative profits: 39.28183467050635
Training window: 2017-08-23/2017-12-07


Training in progress...:   4%|▍         | 2/50 [00:00<00:03, 14.58it/s, sharpe_ratio=-.0622]


Less than 30 observations in the validation set after preprocessing.


### Backtest

In [70]:
version = 1
file_names = [f"rrl_{fees}_{version}" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/no_pca/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}.pkl", "rb"))
    for f in file_names
}

In [71]:
dfs_backtest = {k: create_backtest_dataset(v) for k, v in results.items()}

#### Cumulative profits

In [75]:
initial_invest = 100

In [78]:
buy_and_hold = get_buy_and_hold_df(data.df)
buy_and_hold_sr = calc_sharpe_ratio(returns=buy_and_hold.returns, window_size=252)

In [79]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold, 
    buy_and_hold_sr, 
    initial_invest, 
    layout)

fig.show()

#### Analysis per trading window for one setting

In [80]:
setting = "rrl_no_fees_1"

df = dfs_backtest[setting]
batch_names = np.unique(df["Batch"])

In [81]:
metric ="Cumulative returns"
fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [82]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

#### Portfolio allocation

In [83]:
layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)

fig = plot_avg_portfolio_allocation(results[setting], layout)

fig.show()